# 1. Imports & functions

#### 1.1 Imports

In [ ]:
# Data Wrangling
import pandas as pd
import numpy as np 

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns


#### 1.2 Functions

In [ ]:
# Create New Columns for identifying different guest types
def map_guest_type (row):
    if row["adults"] == 1 and row["children"] == 0 and row["babies"] == 0:
        return "Single"
    elif row["adults"] == 2 and row["children"] == 0 and row["babies"] == 0:
        return "Couple"
    elif (row["adults"] == 2 or row["adults"] == 1) and (row["children"] > 0 or row["babies"] > 0):
        return "Family"
    elif row["adults"] > 2:
        return "Group"
    else:
        return "Other"
        
        

# 2. Load Data

In [ ]:
pwd

In [ ]:
!dir data\\raw

In [ ]:
df = pd.read_csv("../data/raw/hotel_bookings.csv")

# 2. DATA OVERVIEW

In [ ]:
pd.options.display.max_columns = None # show all columns
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df["reserved_room_type"].unique()

#### 2.1 Overview - Missing Values

In [ ]:
missing_values = df.isna().sum()
missing_values = missing_values[missing_values > 0]
missing_values

#### 2.2 Overview - Unique Values

In [ ]:
df.nunique()

#### 2.3 Overview _ Primary visualizations - Guest Type

In [ ]:
df_viz = df.copy()

In [ ]:
df_viz.fillna(0, inplace=True)

In [ ]:
df_viz["guest_type"] = df_viz.apply(map_guest_type, axis=1)

In [ ]:
guest_type_count = df_viz['guest_type'].value_counts()
guest_type_count

In [ ]:
guest_type_count.index

In [ ]:
# Pie Plot (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html)
colors = ['#007acc', '#f48024', '#4caf50', '#d32f2f', '#8e44ad'] # sliceshttps://matplotlib.org/stable/gallery/pie_and_polar_charts/pie_features.html#sphx-glr-gallery-pie-and-polar-charts-pie-features-py

plt.figure(figsize=(8, 8))
plt.pie(guest_type_count, labels=guest_type_count.index, colors=colors, autopct='%1.1f%%', startangle=140) # percentage using autopct: https://stackoverflow.com/questions/6170246/how-do-i-use-matplotlib-autopct
plt.title("Distribution of the different Guest Types")

plt.savefig('../resources/pie_guest_type.png') 

plt.show

# 3. DESCRIPTIVE ANALYSIS 

In [ ]:
df.describe()

# 4. DATA CLEANING

#### 4.1 DATA CLEANING - DROP COLUMNS
Drop Columns not required for analysis

In [ ]:
df_clean = df.copy()

In [ ]:
df_clean.columns

In [ ]:
# Columns to drop

columns_drop_list = ['arrival_date_year',
                    'stays_in_weekend_nights',
                     'stays_in_week_nights',
                     'country',
                     'distribution_channel' , 
                     'distribution_channel' ,
                     'is_repeated_guest' ,
                     'previous_cancellations' ,
                     'previous_bookings_not_canceled' ,
                     'booking_changes' ,
                     'deposit_type' ,
                     'agent' ,
                     'company' ,
                     'customer_type' ,
                     'required_car_parking_spaces' ,
                     'reservation_status' ,
                     'reservation_status_date']

In [ ]:
# Drop Columns
df_clean.drop(columns=columns_drop_list, inplace=True)

In [ ]:
display("n-col-original: ", df.shape[1], "n-col-drop: ", df_clean.shape[1] )

In [ ]:
df_clean.head()

#### 4.1 DATA CLEANING - Handle Missing Values
Deal with mssing values

In [ ]:
missing_values = df_clean.isna().sum()
missing_values = missing_values[missing_values > 0]
missing_values

In [ ]:
df_clean["adults"].unique()

<b style="color:red">Missing Value Strategy</b>
- Get the rows where adults equals two (identify potential parents)
- get mode of children for the parent group
- Replace missing values in children column with the mode calculated for the parent group

In [ ]:
# Check values -> one could already replaces the missing values with 0.0 (why is inter dsplayed as float?)
df_clean[df_clean["adults"] >= 2]["children"].value_counts()

In [ ]:
# get mode
adult_children_mode = df_clean[df_clean["adults"] >= 2].groupby("adults")["children"].agg(lambda x: x.mode()[0])
adult_children_mode

In [ ]:
# map the value of missing values in df_clean children where it maps to the adults columnd in adult_children_mode
missing_children_replace = pd.merge(df_clean[df_clean["children"].isna()], adult_children_mode, left_on="adults", right_index=True, suffixes=('', '_mode'))


In [ ]:
missing_children_replace

In [ ]:
df_clean.loc[df_clean[df_clean["children"].isna()].index, "children"] = missing_children_replace["children_mode"]

In [ ]:
df_clean.isna().sum()

#### 4.2 ADD GUEST TYPE COLUMN column
Add guest_type column to identify different guest types (e.g. single, family, etc.)

In [ ]:
df_clean["guest_type"] = df_clean.apply(map_guest_type, axis=1)

In [ ]:
df_clean["guest_type"].value_counts()

In [ ]:
display("n-col-original: ", df.shape[1], "n-col-drop: ", df_clean.shape[1] )

#### 4.3 Check Guest Type "Others"

In [ ]:
df_clean["adults"][df_clean["guest_type"] == "Other"].value_counts()

In [ ]:
df_clean["children"][df_clean["guest_type"] == "Other"].value_counts()

In [ ]:
df_clean["babies"][df_clean["guest_type"] == "Other"].value_counts()

<b style="color:red">"Others" Strategy</b>
- Get the most frequent value for Adults where Adult is not 0 and children or baby < 0
- Replace the adult value where the adult is 0 and the child or baby is not 0 with the mode value for adults (Assumption: no child or baby stays alone in a hotel -> if there is a child or baby -> there must be an adult)
- Drop remaining rows containing "Other" (adult=0, children=0, babies=0)

In [ ]:
# Get the most frequent value for Adults where Adult is not 0 and children or baby < 0
adult_with_kids_mode = df_clean["adults"][df_clean["adults"]!=0 & ((df_clean["children"] > 0) | (df_clean["babies"] > 0))].mode()[0]
adult_with_kids_mode

In [ ]:
indices = (df_clean["adults"]==0) & ((df_clean["children"] > 0.0) | (df_clean["babies"] > 0))
indices.unique()

In [ ]:
# Replace the adulte value where the adult is 0 and the kid or babie is not 0 with the adult_with_kids_mode var
df_clean.loc[indices, "adults"] = adult_with_kids_mode
df_clean["adults"][df_clean["adults"]!=0 & ((df_clean["children"] > 0) | (df_clean["babies"] > 0))]

In [ ]:
df_clean["adults"][df_clean["guest_type"]=="Other"]

In [ ]:
# Repeant Guest_type Mapping
df_clean["guest_type"] = df_clean.apply(map_guest_type, axis=1)

In [ ]:
df_clean["guest_type"].value_counts()

In [ ]:
df_clean[df_clean["guest_type"]=="Other"].head()

**Update Guest Type Mapping**

In [ ]:
df_clean["guest_type"] = df_clean.apply(map_guest_type, axis=1)

In [ ]:
df_clean[df_clean["guest_type"]=="Other"]

**Drop Columns with remaining "Other" values**

In [ ]:
df_clean.shape

In [ ]:
df_clean = df_clean[df_clean["guest_type"] != "Other"]

In [ ]:
df_clean.shape

**Update Overview _ Primary visualizations - Guest Type**

In [ ]:
df_viz = df_clean.copy()

In [ ]:
df_viz["guest_type"] = df_viz.apply(map_guest_type, axis=1)

In [ ]:
guest_type_count = df_viz['guest_type'].value_counts()
guest_type_count

In [ ]:
guest_type_count.index

In [ ]:
# Pie Plot (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html)
colors = ['#007acc', '#f48024', '#4caf50', '#d32f2f', '#8e44ad'] # sliceshttps://matplotlib.org/stable/gallery/pie_and_polar_charts/pie_features.html#sphx-glr-gallery-pie-and-polar-charts-pie-features-py

plt.figure(figsize=(8, 8))
plt.pie(guest_type_count, labels=guest_type_count.index, colors=colors, autopct='%1.1f%%', startangle=140) # percentage using autopct: https://stackoverflow.com/questions/6170246/how-do-i-use-matplotlib-autopct
plt.title("Distribution of the different Guest Types")

plt.savefig('../resources/pie_guest_type.png') 

plt.show

#### 4.4 CONVERT MONTH NAMES TO CORRESPONDING NUMERIC REPRESENTATION

In [ ]:
#### 4.5 WRITE CLEANED DATASET TO FOLDER

In [ ]:
df_clean["arrival_date_month"] = pd.to_datetime(df_clean["arrival_date_month"], format="%B").dt.strftime('%m') # https://www.geeksforgeeks.org/python-datetime-strptime-function/


In [ ]:
df_clean["arrival_date_month"].unique()

#### 4.5 WRITE CLEANED DATASET TO FOLDER

In [ ]:
df_clean.to_csv('../data/cleaned/hotel_bookings_cleaned.csv')

# 5.DATA INSIGHTS

#### 5.1 Peak Time of Hotel Bookings By Guest Type

In [ ]:
df_in = df_clean.copy()

In [ ]:
df_in.columns

In [ ]:
df_in["arrival_date_month"].unique()

In [ ]:
# Count arrivals
arrival_counts = df_in.groupby(["arrival_date_month", "guest_type"])["adults"].count().unstack()
arrival_counts

In [ ]:
colors = {"Couple": "#007acc", 
          "Single": "#f48024", 
          "Family": "#4caf50", 
          "Group": "#d32f2f", 
          "Other": "#8e44ad"}



sns.set_style("whitegrid")

plt.figure(figsize=(10,6))
sns.lineplot(data=arrival_counts, dashes=False, palette=colors)

plt.title("Arrival of Different Guest Types Over Months")
plt.xlabel("Month")
plt.ylabel("Number of Arrivals")

plt.legend(title="Guest Type")

plt.savefig('../resources/arrival_by_guest_over_time.png') # Save as PNG Image to resources


plt.show()

In [ ]:
# Normalize data
# https://stackoverflow.com/questions/26414913/normalize-columns-of-a-dataframe
arrival_counts_norm = (arrival_counts - arrival_counts.min()) / (arrival_counts.max() - arrival_counts.min())
arrival_counts_norm

In [ ]:
colors = {"Couple": "#007acc", 
          "Single": "#f48024", 
          "Family": "#4caf50", 
          "Group": "#d32f2f", 
          "Other": "#8e44ad"}

sns.set_style("whitegrid")

plt.figure(figsize=(10,6))
sns.lineplot(data=arrival_counts_norm, dashes=False, palette=colors)

plt.title("Arrival of Different Guest Types Over Months (Normalized)")
plt.xlabel("Month")
plt.ylabel("Number of Arrivals")

plt.legend(title="Guest Type")

plt.savefig('../resources/arrival_by_guest_over_time_norm.png') # Save as PNG Image to resources


plt.show()

#### 5.2 Average Daily Rate By Customer

Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights

In [ ]:
adr_guest_type = df_in.groupby("guest_type")["adr"].mean()
adr_guest_type

In [ ]:
# Visualize ADR by guest type
colors = {"Couple": "#007acc", 
          "Single": "#f48024", 
          "Family": "#4caf50", 
          "Group": "#d32f2f", 
          "Other": "#8e44ad"}

# Sort Values descending
adr_guest_type_sorted = adr_guest_type.sort_values(ascending=False)

# Create Dataframe to be able to map the colors to the sorted values
adr_guest_type_sorted = adr_guest_type_sorted.to_frame().reset_index()
adr_guest_type_sorted['color'] = adr_guest_type_sorted["guest_type"].map(colors)

# Create Bar Plot
plt.figure(figsize=(10,6))
plt.bar(adr_guest_type_sorted["guest_type"], adr_guest_type_sorted["adr"], color=adr_guest_type_sorted["color"])


plt.title('ADR by Guest Type')
plt.xlabel("Guest Type")
plt.ylabel("ADR")

plt.tight_layout()

plt.savefig('../resources/adr_by_guest.png') # Save as PNG Image to resources

plt.show() # Show Graph

<b style="color:green"> ADR BY Guest Type - Findings</b></br>

- We see differences in the ADR between the different groups
      
- **Families && Groups**:
    - Higher ADR values compared to others -> Willing to spend more
    - Due to larger group size or preferences for more costly accomodations


- **Couples, Singles and Others**:
    - Have a lower ADR with Singles having the lowest ADR
    - Singles may tend to more budget-friendly accomodations
    - Others define an unkown group with a significant ADR 




<b style="color:green"> ADR - Reommendations</b></br>

- **Tailored Offerings**: Develop tailored offerings for the different groups specifically for the "Group" and "Family" types to gain from their willingnes to spend more
- **Price Optimization**: E.g. Offer special pricing for "Singles" or target these groups for sales purposes
- **Further Analysis**: Gather further analysis on why the different guest types have different ADRs 



#### 5.3 Average Daily Rate By Customer Over Time [Months]

Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights

In [ ]:
df_in.columns

In [ ]:
df_in["arrival_date_month"].nunique()

In [ ]:
# Sort data
adr_guest_type_date = df_in.sort_values("arrival_date_month")


colors = {"Couple": "#007acc", 
          "Single": "#f48024", 
          "Family": "#4caf50", 
          "Group": "#d32f2f", 
          "Other": "#8e44ad"}


sns.set_style("whitegrid")

plt.figure(figsize=(10,6))
sns.lineplot(data=adr_guest_type_date, x="arrival_date_month", y="adr", hue="guest_type", palette=colors)

plt.title("ADR per Guest Type Over Time [Months]")
plt.xlabel("Month")
plt.ylabel("ADR")

plt.legend(title="Guest Type")

plt.savefig('../resources/adr_per_guest_over_time.png') # Save as PNG Image to resources


plt.show()


#### 5.4 Room type preferences By Guest Type

In [ ]:
# Room Types
df_in["reserved_room_type"].unique()

In [ ]:
room_type_pref = df_in.groupby("guest_type")['reserved_room_type'].value_counts().unstack() # see usntack: 2.3_combining_structuring_data
room_type_pref

#Normalize (https://stackoverflow.com/questions/52007165/normalizing-rows-of-pandas-dataframe)
room_type_pref_norm = room_type_pref.div(room_type_pref.sum(axis=1), axis=0)
room_type_pref_norm

In [ ]:
# Visualize room type preference  by guest type

room_type_pref_norm.plot(kind='bar', figsize=(10, 6)) # https://stackoverflow.com/questions/11927715/how-to-give-a-pandas-matplotlib-bar-graph-custom-colors
plt.title('Room Preferences by Guest Type (Normalized)')
plt.xlabel("Guest Type")
plt.ylabel("Number of Bookings")
plt.legend(title='Room Type', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust y-axis to better dispaly the lower value bars
plt.ylim(0, 1)

plt.tight_layout()

plt.savefig('../resources/room_type_pref.png') # Save as PNG Image to resources

plt.show() # Show Graph

<b style="color:green"> Room Type Preferences - Findings</b></br>

- **Couples**:
    - Room Type A is the most preferred among couples, with significantly higher number of bookings compared to other room types.
    - Only few bookings for room types "H" and "L"
      
- **Families**:
    - Room Type A is the most preferred among families, but not as striking. Followed by "F" and "G"
    - There are als few bookings for room types "H" and "L"
      
 - **Groups**:
    - Room Type D is the most preferred among Groups, but not as striking. Followed by "A".
        - The other room types are insignificant compared to the others

- **Others**:
    - Rather an insignifcant group -> re-allocate?
 
- **Singles**:
    - Room Type A is the most preferred among Groups, with significantly higher number of bookings compared to other room types.
    - Followed by "D".
    - The other room types are insignificant compared to the others




<b style="color:green"> Room Type Preferences - Reommendations</b></br>

- **Optimize Room Allocation**: Focus availability of preferred room types
- **Include Special Promotions**: Offer special promotions or discounts for less popular room types
- **Feedback**: Include further investigations on why certain groups prefer certain room types



#### 5.5 Meals Package Choice By Guest Type

In [ ]:
df_in["meal"].unique()

**Abbreviations** 

<a href="https://support.travelrepublic.co.uk/Before-I-Book/BIB-Accommodation/1035827372/What-does-the-board-basis-mean.htm">Hotel Meals Explained</a>

- **O** (Room Only): No meals are included.
- **SC** (Self Catering): No meals are included; however, your accommodation will be provided with catering facilities for you to cook light meals.
- **BB** (Bed and Breakfast): Breakfast is included.
- **HB** (Half Board): Breakfast and evening meals are included. In some cases, you can choose to receive lunch instead of breakfast – the hotel will confirm this on arrival.
- **FB** (Full Board): Breakfast, lunch and evening meals are included.

In [ ]:
meal_pref = df_in.groupby("guest_type")["meal"].value_counts().unstack()
meal_pref

#Normalize (https://stackoverflow.com/questions/52007165/normalizing-rows-of-pandas-dataframe)
meal_pref_norm = meal_pref.div(room_type_pref.sum(axis=1), axis=0)
meal_pref_norm


In [ ]:
# Visualize meal type preference by guest type

meal_pref_norm.plot(kind='bar', figsize=(10, 6)) # https://stackoverflow.com/questions/11927715/how-to-give-a-pandas-matplotlib-bar-graph-custom-colors
plt.title('Meal Preferences by Guest Type (Normalized)')
plt.xlabel("Guest Type")
plt.ylabel("Number of Orderings")
plt.legend(title='Meal Type', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust y-axis to better dispaly the lower value bars
plt.ylim(0, 1)

plt.tight_layout()

plt.savefig('../resources/meal_type_pref.png') # Save as PNG Image to resources

plt.show() # Show Graph

<b style="color:green"> Meal Type Preferences - Findings</b></br>

- **Bed and Breakfast (BB)**:
    - Popular among all guest types, with a significant number of bookings for couples and singles.
    - A meal option included seems to be preffered as HB (half )
      
- **Half Board (HB) and Self-Catering (SC)**:
    - Couples show a preference for these meal type, with a significant number of bookings
    - Relativle insignificant for other guest types
      
 - **Others**:
    - Other meal types are less interesing



<b style="color:green"> Meal Type Preferences - Reommendations</b></br>

- **Customization**: Customize meal options or allow for customization options for meal packages
- **Promote Meal Packages**: Promote meal packages such as HB or FB to increase revenue
- **Feedback**: Include further investigations on why certain groups prefer certain meal types


#### 5.6 No of special requests by Guest type

Number of special requests made by the customer (e.g. twin bed or high floor)

In [ ]:
df["total_of_special_requests"].unique()

In [ ]:
spec_req = df_in.groupby("guest_type")["total_of_special_requests"].value_counts().unstack()
spec_req
#Normalize (https://stackoverflow.com/questions/52007165/normalizing-rows-of-pandas-dataframe)
spec_req_norm = spec_req.div(spec_req.sum(axis=1), axis=0)

spec_req_norm

In [ ]:
# Visualize meal type preference by guest type

spec_req_norm.plot(kind='bar', figsize=(10, 6)) # https://stackoverflow.com/questions/11927715/how-to-give-a-pandas-matplotlib-bar-graph-custom-colors
plt.title('No of Special Requests by Guest Type (Normalized)')
plt.xlabel("Guest Type")
plt.ylabel("Number of Special Requests")
plt.legend(title='No of requests', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust y-axis to better dispaly the lower value bars
plt.ylim(0, 1)

plt.tight_layout()

plt.savefig('../resources/no_spec_req.png') # Save as PNG Image to resources

plt.show() # Show Graph

#### 5.7 Cancellation Rate by Guest Type and Room Type Match

The Cancelation Rate is matched to the numbers of matches between reserved rooms and actually assigned rooms.

In [ ]:
df["booking_changes"].unique()

In [ ]:
df["days_in_waiting_list"].unique()

In [ ]:
df["reserved_room_type"].unique()

In [ ]:
# Create a new column indicating whether reserved and assigned room types match
df_clean['room_type_match'] = (df_clean['reserved_room_type'] == df_clean['assigned_room_type']).astype(int)
df_clean['room_type_match']

In [ ]:
# Calculate the cancellation rate for each combination of room type match and guest type
cancellation_rates = df_clean.groupby(['guest_type', 'room_type_match'])['is_canceled'].mean().reset_index()
cancellation_rates

In [ ]:
# Plot the cancellation rates for each guest type
plt.figure(figsize=(12, 8))
sns.barplot(data=cancellation_rates, x='guest_type', y='is_canceled', hue='room_type_match', palette='colorblind')
plt.title('Cancellation Rate by Guest Type and Room Type Match')
plt.xlabel('Guest Type')
plt.ylabel('Cancellation Rate')
plt.legend(title='Room Type Match', loc='upper right')
plt.xticks(rotation=45)

plt.savefig('../resources/cancel_rate_room_type_match.png')

plt.show()